# A/B Test for Marketing Campaigns
### Vu Duong
#### June, 2020

# CREDITS
This work is inspired by multiple greate sources done before:

- https://towardsdatascience.com/hypothesis-testing-in-machine-learning-using-python-a0dc89e169ce
- https://towardsdatascience.com/inferential-statistics-series-t-test-using-numpy-2718f8f9bf2f
- https://rstudio-pubs-static.s3.amazonaws.com/365114_c8b996abd737463fa7f952e96ab34480.html
- https://www.ibm.com/communities/analytics/watson-analytics-blog/marketing-campaign-eff-usec_-fastf/

# INTRODUCTION
- Analyze marketing campaign based on sales, location, and other key metrics
- Figure out which channels, campaigns, or anything help us predict sale from customers 

# Scenario
A fast food chain plans to add a new item to its menu. However, they are still undecided between three possible marketing campaigns for promoting the new product. In order to determine which promotion has the greatest effect on sales, the new item is introduced at locations in several randomly selected markets. A different promotion is used at each location, and the weekly sales of the new item are recorded for the first four weeks.

Our first major focus on 3 size of Market campaign:
- Small
- Medium
- Large

Our second major focus on 3 promotional strategies:
1. Collectibles - Fast food companies can drive sales through collectibles, particularly those that kids enjoy. For instance, a popular animated film, dolls, glasses or other mementos that are related to the movie. The owners decided to provide one free item for the purchase of the vegan patty burger. This fast food marketing strategies entices people to come back until they have all the collectibles.

2. Societal Marketing - It includes volunteering or collecting money or items for charity. Consumers who relate to our ideas and values due to our charitable work may, in turn, patronize our fast food restaurant.

3. Loyalty Programs - Frequency card programs are a popular type of loyalty program for fast food restaurants. The owners plan to invite people to fill out an application and reward them according to the frequency in which they order the particular product. They planned on giving a free drink after their first four orders, then free fries after their next four orders. Ultimately, a customer could earn a free meal after 12 orders.

Detailed description of dataset content is described in the following link:
https://www.ibm.com/communities/analytics/watson-analytics-blog/marketing-campaign-eff-usec_-fastf/

# LIBRARY

In [ ]:
# Data Processing
import numpy as np
import pandas as pd

# Data Visualizing
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator
from IPython.display import display, HTML

# Math
from scipy import stats  # Computing the t and p values using scipy 
from statsmodels.stats import weightstats 

# Warning Removal
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

# DATA EXPLORATION

In [ ]:
df = pd.read_csv('../input/WA_Fn-UseC_-Marketing-Campaign-Eff-UseC_-FastF.csv')

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.info()

### Analysis of the Target feature (SaleInThousands)

#### MarketID and MarketSize
- Any MarketID is accompanied by only one MarketSize
- MarketID with MarketSize of Large and Small place on the top Sales, while those with MarketSize of Medium come last.

In [ ]:
pd.DataFrame(df.groupby(['MarketID', 'MarketSize'])['SalesInThousands'].mean().sort_values(ascending=False))

#### MarketSize and A Number of Campains in each MarketSize and SaleInThousands
- MarketSize with Large label account for most Sale, followed by Small and the last is Medium size. Here is to comfirm the statement above.
- The 1st image(Sales in Thousands) below show Sales of 41% comes from Large, of 33% from Small, only of 26% from Medium MarketSize.
- With the 2nd image, most sale is from the Medium MarketSize.
- Yet, the 3nd image(Percentage of a number of campaigns) shows a number of Meidum campaigns outnumber the sum of Large and Small campaigns, while contributing small amount of Sales in total.

In [ ]:
pd.DataFrame(df.groupby(['MarketSize'])['SalesInThousands'].mean().sort_values(ascending=False))

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(20, 6))
grid = gridspec.GridSpec(nrows=1, ncols=3,  figure=fig)

ax1 = fig.add_subplot(grid[0, 0])
df.groupby(['MarketSize'])['SalesInThousands'].mean().plot.pie(autopct='%1.0f%%', ax=ax1)
plt.ylabel('')
plt.title('Average Sales In Thousands')

ax1 = fig.add_subplot(grid[0, 1])
df.groupby(['MarketSize'])['SalesInThousands'].sum().plot.pie(autopct='%1.0f%%', ax=ax1)
plt.ylabel('')
plt.title('Sum of Sales In Thousands')

ax2 = fig.add_subplot(grid[0, 2])
df.groupby(['MarketSize'])['MarketID'].count().plot.pie(autopct='%1.0f%%', ax=ax2)
plt.ylabel('')
plt.title('Percentage of a number of campaigns')
plt.show()

#### MarketSize and AgeOf Store
- Seeing that Large MarketSize focus on young stores, followed by Medium, and Small at last.
- One possible reason to explain why Large and Small account for a large of amount Sales is people tend to visit the newest and fanciest stores to enjoy their life and to determine their social status while people also visit their daily stores so as to get nostalgic and feel at home. 

In [ ]:
df.groupby(['MarketSize'])['AgeOfStore'].agg(['mean','median']).sort_values(by=['mean'], ascending=False)

#### Promotion
- Promotion 1 and 3 are quite similar in term of SalesInThousands, yet different from Promotion 2. We will explore using Z Test and F Test (Anova). 
- Combining both MarketSize and Promotion indicates the same results as a statement above.
- Calculating a number of marketing campaigns of each MarketSize for each Promotion shows Medium campaigns is the highest, followed by Large and then Small. It makes sense as Small MarketSize concentrating on old stores where people knew really well. Large MarketSize concentrating on new stores where people barely knew, so if marketing those places would generate huge profit. However Medium MarketSize received a lot of marketing campaign but return a little revenue.

In [ ]:
pd.DataFrame(df.groupby(['Promotion'])['SalesInThousands'].mean().sort_values(ascending=False))

In [ ]:
pd.DataFrame(df.groupby(['MarketSize', 'Promotion'])['SalesInThousands'].mean().sort_values(ascending=False))

In [ ]:
# Now let's view the marketsize for each promotion
pd.DataFrame(df.groupby(['MarketSize', 'Promotion'])['MarketID'].count())

In [ ]:
df.groupby(['MarketSize', 'Promotion']).count()['MarketID'].unstack('MarketSize')

In [ ]:
df.groupby(['MarketSize', 'Promotion']).count()['MarketID']

In [ ]:
plt.figure(constrained_layout=True, figsize=(10, 8))
sns.countplot(x=df['Promotion'], hue=df['MarketSize'])

In [ ]:
df.groupby(['MarketSize', 'Promotion']).count()['MarketID'].unstack('MarketSize').plot(
                                                                                    kind='bar',
                                                                                    figsize=(12,10),
                                                                                    grid=True,
                                                                                    stacked=True)

#### AgeOfStore
- Most Age of Stores cluter on the left side.

In [ ]:
sns.distplot(df['AgeOfStore'])

In [ ]:
sns.countplot(df['AgeOfStore'])

#### Week
- It seems like there are no differences among 4 weeks of running campaign
- Making more depth of week analysis based on Promotion and MarketSize, yet nothing seems to change.

In [ ]:
pd.DataFrame(df.groupby(['week'])['SalesInThousands'].mean().sort_values(ascending=False))

In [ ]:
pd.DataFrame(df.groupby(['Promotion', 'week'])['SalesInThousands'].mean().sort_values(ascending=False))

In [ ]:
pd.DataFrame(df.groupby(['MarketSize', 'week'])['SalesInThousands'].mean().sort_values(ascending=False))

# Performing A/B Testing

### Observation
- As we can see diagram cross 3 Promotions, all of which follow quite a shape of Bell Curve.

In [ ]:
fig = plt.figure()
sns.distplot(df.loc[df['Promotion'] == 1, 'SalesInThousands'])
plt.title('Promotion 1')
plt.xlabel('SaleInThousand')


fig = plt.figure()
sns.distplot(df.loc[df['Promotion'] == 2, 'SalesInThousands'])
plt.title('Promotion 2')
plt.xlabel('SaleInThousand')

fig = plt.figure()
sns.distplot(df.loc[df['Promotion'] == 3, 'SalesInThousands'])
plt.title('Promotion 3')
plt.xlabel('SaleInThousand')

### F-Test and Z-Test overview
- Threshold that reject the Null is usually less than 0.05.
- P-Value is close to 0 which suggests that there is good evidence to REJECT the Null Hypothesis, leading a statistical difference between the two groups.
- Sample size is greater than 30. Otherwise, use a t test.
- Data points should be independent from each other. In other words, Promotion 1 does not affect Promotion 2 or 3.
- Our data should look normally distributed. However, for large sample sizes (over 30) this doesn’t always matter.
- To deal with the similarity between 2 or more groups of Promotion at the same time, we use ANOVA (F-TEST).
- Next, to know which Promotion is a statistically different from the other, there are 2 types of Z tests: one-sample Z test and two-sample Z test that handle the need. Here we are using two-sample Z test for comparison of among 3 Promotions.

#### ANOVA (F-TEST)

##### Obvervation
- Not all promotions are the same, as p-value suggest a value less than 0.05.

In [ ]:
# a group of Promotions
PromotionNumber = df['Promotion'].unique()
# pd.unique(df['Promotion'])

# a list of sales are categorized according to Promotion group, using dict
d_data = {promotion:df[df['Promotion'] == promotion]['SalesInThousands'] for promotion in PromotionNumber}

# apply Anova to 3 groups
F, p = stats.f_oneway(d_data[1], d_data[2], d_data[3])
print("p-value: {}, thus rejecting the null hypothesis".format(p))

#### Z TEST

##### Obvervation
- We figure out which promotion is different from others.
- p-value between promotion 1 and 2 is so small, indicating a strong evidence against null hypothesis.
- p-value between promotion 1 and 3 is greater than 0.05, indicating a strong evidence of null hypothesis.
- p-value between promotion 2 and 3 is so small, indicating a strong evidence against null hypothesis.
- We know promotion 1 and 3 outperform promotion 2, and the difference between  promotion 1 and 3 is not statistically different.

In [ ]:
t, p = stats.ttest_ind(df.loc[df['Promotion'] == 1, 'SalesInThousands'],
                       df.loc[df['Promotion'] == 2, 'SalesInThousands'], 
                       equal_var=False)
print("p-value = {:.4f}, thus Promotion 1 and 2 are statistically similar".format(p))

t, p = stats.ttest_ind(df.loc[df['Promotion'] == 1, 'SalesInThousands'],
                       df.loc[df['Promotion'] == 3, 'SalesInThousands'], 
                       equal_var=False)
print("p-value = {:.4f}, thus Promotion 1 and 3 are statistically different".format(p))

t, p = stats.ttest_ind(df.loc[df['Promotion'] == 2, 'SalesInThousands'],
                       df.loc[df['Promotion'] == 3, 'SalesInThousands'], 
                       equal_var=False)
print("p-value = {:.4f}, thus Promotion 2 and 3 are statistically similar".format(p))

# MODELING

### Observation
- So far, we have analyzed a lot on MarketSize and Promotion.
- To comfirm those statements above are right, we put all features in linear model to see which features are crucial that Sale is affected by those.
- As the graph below shows MarketSize and Promotion are leading in features importance when it comes to Sale Prediction.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
df['MarketID'] = df['MarketID'].astype(int)
df['Promotion'] = df['Promotion'].astype(str)
all_data = pd.get_dummies(df.drop('LocationID', axis=1), drop_first=True)

In [ ]:
x = all_data.drop('SalesInThousands', axis=1)
y = all_data['SalesInThousands']

In [ ]:
reg = LinearRegression().fit(x, y)
coef = pd.Series(reg.coef_, index = x.columns)

imp_coef = coef.sort_values()
imp_coef.plot(kind = "barh")
plt.title("Feature importance using Linear Model")

# Result from the Study
- A particular MarketSize appear with 1 LocationID
- Most Sale come from medium and large market which receive a large number of marketing campaign as well. 
- However the average sale is more important to consider when it show that Large and Small MarketSize display a significant sales.
- Promotion 1 and 3 are quite at the same sales figure level.
- The majority of store age is young.
- Sale in all weeks are the almost same.
- Conducting A/B Test on Promotion to find out if all promotions are alike.
- Modeling the whole dataset in linear regression model as a way to comfirm all statements above is right.